Defining paths and labels

In [ ]:
import os
from PIL import Image
import numpy as np

train_path = './inputs/chest_Xray/train'
val_path = './inputs/chest_Xray/val'
test_path='./inputs/chest_Xray/test'
types = ['NORMAL','PNEUMONIA']
#`type_to_label` is a dictionary that maps each type of image (NORMAL or PNEUMONIA) to a numerical label (0 or 1). The line `type_to_label = {type: i for i, type in enumerate(types)}` uses a dictionary comprehension to create this mapping by iterating over the `types` list and assigning a numerical label to each type using the `enumerate` function.
type_to_label = {type: i for i, type in enumerate(types)}

print(types)

Load and process image

In [ ]:
def load_and_preprocess_image(imagePath,target_size=(224, 224)):
    img = Image.open(imagePath).convert('RGB')
    img_resized = img.resize(target_size)
    img_array = np.array(img_resized)
    img_normalized = img_array / 255.0  # Normalize pixel values between 0 and 1
    return img_normalized

Load the training images and labels

In [ ]:
train_images = []
train_labels = []
for type in types:
    type_path = os.path.join(train_path, type)
    for image_name in os.listdir(type_path):
        if image_name.endswith('.jpeg'):
            image_path = os.path.join(type_path, image_name)
            train_images.append(load_and_preprocess_image(image_path))
            train_labels.append(type_to_label[type])
train_images = np.array(train_images)
# train_images = train_images.astype(np.float32)  # Convert the data type to float32
train_images = np.expand_dims(train_images, axis=-1)
train_labels = np.array(train_labels)

Load the validation images and labels

In [ ]:

val_images = []
val_labels = []
for type in types:
    type_path = os.path.join(val_path, type)
    for image_name in os.listdir(type_path):
        if image_name.endswith('.jpeg'):
            image_path = os.path.join(type_path, image_name)
            val_images.append(load_and_preprocess_image(image_path))
            val_labels.append(type_to_label[type])
        
val_images = np.array(val_images)
val_images = np.expand_dims(val_images, axis=-1)
val_labels = np.array(val_labels)


Load the evaluation images and labels

In [ ]:
test_images = []
test_labels = []
for type in types:
    type_path = os.path.join(test_path, type)
    for image_name in os.listdir(type_path):
        if image_name.endswith('.jpeg'):
            image_path = os.path.join(type_path, image_name)
            test_images.append(load_and_preprocess_image(image_path))
            test_labels.append(type_to_label[type])
                      
test_images = np.array(test_images)
test_images = np.expand_dims(test_images, axis=-1)
test_labels = np.array(test_labels)

Define the CNN model

In [ ]:
import tensorflow as tf
from tensorflow import keras
# Define the CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.AveragePooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='softmax')  # Change the number of units and activation function
])

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Train the model on the training data with early stopping and model checkpointing
callbacks = [keras.callbacks.EarlyStopping(patience=3),
             keras.callbacks.ModelCheckpoint('pneumonia.h5', save_best_only=True)]
model.fit(train_images, train_labels, epochs=20,batch_size=32 ,validation_data=(val_images, val_labels), callbacks=callbacks)


Evalute the model with evaluation data

In [ ]:
model.load_weights('pneumonia.h5')
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Predict disease

In [ ]:
import numpy as np
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('pneumonia.h5')

img = load_and_preprocess_image('./inputs/chest_Xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg')
img = np.expand_dims(img, axis=0)
# Perform the prediction
pred = model.predict(img)
print(pred)
pred_type = "NORMAL" if pred > 0 else "PNEUMONIA"

# print('The predicted type is: ' + pred_type)